### **title 기반 Smart Genre Relevance Score (지능형 장르 적합도) 설계**

GPT가 의도한 장르랑 실제 추천된 노래의 장르가 적합한지

오디오속성적합도 -> 스포티파이 권한 이슈로 불가 -> 장르적합도 -> 스포티파이가 각 노래 별 장르 제공 안해줌, 아티스트 별 장르만 제공함 -> 아티스트 장르 기반 적합도 -> GPT의 타켓장르(키워드)랑 실제 아티스트 장르와의 매치가 딱 들어맞기 어려워 fail이 너무 많이 발생함+ 일일이 룰을 짜는 건 한계가 있음 -> 데이터소스(스포티파이 아티스트 장르)는 객관적 사실을 쓰되, 판단은 AI를 쓰는 하이브리드 방식 -> **LLM 기반 장르 적합도 평가**(유지보수가 필요 없음)

- **Goal :** 접근 가능한 '아티스트 장르(Artist Genres)' 데이터(Fact)와 LLM의 추론 능력을 결합하여, GPT 추천 결과가 상황에 논리적으로 부합하는지 검증하는 대리 지표(Proxy Metric) 도입. 평가의 핵심 근거가 LLM의 상상이 아닌 **Spotify 데이터베이스(Fact)**에서 오므로 객관적 근거를 가짐.

- **Method**

접근 방식: 외부 데이터 기반 검증 (External Data Verification)

1. GPT 의도 파악 (Intent): GPT가 추천 시 설정한 `Primary Tag`(예: focus_instrumental)와 `Reasoning`을 추출.
2. Ground Truth 수집 (Fact): Spotify API를 통해 해당 아티스트의 공식 `Genres` 리스트(예: ['soundtrack', 'piano'])를 조회.
3. LLM 판단 (Judgment): 수집된 **실제 장르(Fact)**가 의도한 분위기(Intent) 범주에 논리적으로 포함되는지 LLM(Judge)이 판정.
    - *질문 예시: "실제 장르인 'Death Metal'이 의도한 'Sleep' 분위기에 적합한가?" → FAIL*

- **Target** 

목표치: 90% 이상 (Pass Rate)

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from openai import OpenAI
import json
import time

# ===========================================================
# 1. 설정
# ===========================================================
SPOTIFY_CLIENT_ID = ""
SPOTIFY_CLIENT_SECRET = ""
OPENAI_API_KEY = ""

INPUT_FILE = "search_hit_rate_result.xlsx"
OUTPUT_FILE = "genre_relevance_final.xlsx"

# ===========================================================
# 2. 연결 (Spotify & OpenAI)
# ===========================================================
try:
    auth_manager = SpotifyClientCredentials(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET
    )
    sp = spotipy.Spotify(auth_manager=auth_manager)
    client = OpenAI(api_key=OPENAI_API_KEY)
    print(" API 연결 성공! (Smart Genre Check Mode)")
except Exception as e:
    print(f" 연결 실패: {e}")
    exit()

# (여기에 있던 SYSTEM_PROMPT_DJ는 필요 없어서 삭제했습니다!)
# 평가는 엑셀에 있는 데이터를 읽어서 하는 것이니까요.

# ===========================================================
# 3. LLM에게 장르 적합성 판단 맡기기
# ===========================================================
def check_genre_fit_with_llm(artist_name, actual_genres, primary_tag, scenario):
    """
    어휘 제한된 Primary Tag (예: focus_piano)를 파싱하여
    실제 장르와 비교하는 스마트 검증 함수.
    """
    # 장르 정보가 없으면 판단 불가
    if not actual_genres:
        return "UNKNOWN", "No genre data on Spotify"

    # 1. 태그 파싱: "focus_piano" -> "piano" 추출
    # (만약 포맷이 안 지켜져서 '_'가 없으면 그냥 통째로 씀)
    try:
        if '_' in str(primary_tag):
            target_genre = str(primary_tag).split('_')[-1] # 뒤쪽 단어(Genre)만 씀
        else:
            target_genre = str(primary_tag)
    except:
        target_genre = str(primary_tag)

    # 2. LLM에게 질문
    prompt = f"""
    You are a Music Genre Classifier.
    
    # Input Data
    - Target Genre (AI Prediction): "{target_genre}"
    - Actual Spotify Genres (Fact): {actual_genres}
    - Artist Name: "{artist_name}"
    - Scenario: "{scenario}"

    # Task
    Check if the 'Actual Spotify Genres' logically verify the 'Target Genre'.
    
    # Rules
    - PASS: If the Target Genre is present in or strongly related to Actual Genres.
      (e.g., Target='piano' AND Actual=['classical piano', 'composer'] -> PASS)
    - FAIL: If they are completely different or contradictory.
      (e.g., Target='piano' AND Actual=['death metal'] -> FAIL)
    
    # Output Format (JSON)
    {{"result": "PASS" or "FAIL", "reason": "short reason"}}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"},
            temperature=0.0
        )
        res_json = json.loads(response.choices[0].message.content)
        return res_json['result'], res_json['reason']
    except Exception as e:
        return "ERROR", str(e)

# ===========================================================
# 4. 메인 실행
# ===========================================================
print(f"\n Smart Genre Relevance 분석 시작... (파일: {INPUT_FILE})")

try:
    df = pd.read_excel(INPUT_FILE)
except:
    print(" 파일을 찾을 수 없습니다.")
    exit()

results = []
pass_count = 0
analyzed_count = 0

for index, row in df.iterrows():
    track_id = row.get('Spotify ID')
    search_status = row.get('Spotify Search')
    
    # 검색 실패 등은 건너뜀
    if search_status != 'SUCCESS' or pd.isna(track_id):
        results.append({**row, "Genre Status": "SKIPPED"})
        continue
        
    analyzed_count += 1
    
    try:
        # 1. Spotify에서 장르 가져오기 (Fact)
        track_info = sp.track(track_id)
        artist_id = track_info['artists'][0]['id']
        artist_info = sp.artist(artist_id)
        actual_genres = artist_info.get('genres', []) 
        
        # 2. 엑셀에 저장된 Primary Tag 가져오기
        primary_tag = row.get('Primary Tag', 'Unknown')
        scenario = row.get('Scenario', '')
        
        print(f" Analyzing: {row['Real Artist']} (Tag: {primary_tag}) ...", end=" ")

        # 3. LLM에게 판단 요청 (파싱은 함수 안에서 자동 수행)
        status, reason = check_genre_fit_with_llm(row['Real Artist'], actual_genres, primary_tag, scenario)
        
        if status == "PASS":
            pass_count += 1
            print("✅ PASS")
        else:
            print(f"❌ FAIL ({reason})")

        results.append({
            **row,
            "Actual Genres": str(actual_genres),
            "Genre Status": status,
            "Genre Reason": reason
        })

    except Exception as e:
        print(f" Error: {e}")
        results.append({**row, "Genre Status": "ERROR"})
        
    time.sleep(0.3) 

# 결과 저장
result_df = pd.DataFrame(results)
result_df.to_excel(OUTPUT_FILE, index=False)

score = (pass_count / analyzed_count) * 100 if analyzed_count > 0 else 0

print("\n" + "="*50)
print(f" 분석 완료! 파일: {OUTPUT_FILE}")
print(f" [Smart KPI] Genre Relevance Score: {score:.2f}%")
print("="*50)

 API 연결 성공! (Smart Genre Check Mode)

 Smart Genre Relevance 분석 시작... (파일: search_hit_rate_result.xlsx)
 Analyzing: Ludovico Einaudi (Tag: focus_classical) ... ✅ PASS
 Analyzing: AC/DC (Tag: active_rock) ... ✅ PASS
 Analyzing: Max Richter (Tag: sleep_ambient) ... ✅ PASS
 Analyzing: Miles Davis (Tag: relax_jazz) ... ✅ PASS
 Analyzing: NewJeans (Tag: active_k-pop) ... ✅ PASS
 Analyzing: Derrol (Tag: focus_lofi) ... ✅ PASS
 Analyzing: Dua Lipa (Tag: neutral_pop) ... ✅ PASS
 Analyzing: IU (Tag: consolation_k-pop) ... ✅ PASS
 Analyzing: Metallica (Tag: focus_metal) ... ✅ PASS
 Analyzing: Ludwig van Beethoven (Tag: active_classical) ... ✅ PASS
 Analyzing: J. Cole (Tag: sleep_hip-hop) ... ✅ PASS
 Analyzing: BTS (Tag: focus_k-pop) ... ✅ PASS
 Analyzing: ODESZA (Tag: relax_electronic) ... ✅ PASS
 Analyzing: Foo Fighters (Tag: consolation_rock) ... ✅ PASS
 Analyzing: Johann Sebastian Bach (Tag: focus_classical) ... ✅ PASS
 Analyzing: Rage Against The Machine (Tag: anger_rock) ... ✅ PASS
 Analyzi